In this assignement, feel free to use the `sparse` module from `scipy`.

Use the cell below for your imports.

In [42]:
import numpy as np
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix

implement the function `mat_mul_coo` that takes two sparse matrices in `coo` and returns their product.

In [43]:
import numpy as np
from scipy.sparse import coo_matrix

def mat_mul_coo(matrix1, matrix2):
    #Dimensions compatiblity
    if matrix1.shape[1] != matrix2.shape[0]:
        raise ValueError("Different shapes")
    
    # Convert the matrices to dictionaries for easier manipulation
    mat1dict = {(matrix1.row[i],matrix1.col[i]): matrix1.data[i] for i in range(len(matrix1.data))}
    mat2dict = {(matrix2.row[i],matrix2.col[i]): matrix2.data[i] for i in range(len(matrix2.data))}
    
    # The product of the two matrices which will be stored in mat3dict
    mat3dict = {}
    for key1, val1 in mat1dict.items():
        for key2, val2 in mat2dict.items():
            if key1[1] == key2[0]:
                if (key1[0],key2[1]) in mat3dict:
                    mat3dict[(key1[0],key2[1])] += val1*val2
                else:
                    mat3dict[(key1[0],key2[1])] = val1*val2
    
    # Convert the resulting dictionary to COO format
    row = []
    col = []
    data = []
    for key, val in mat3dict.items():
        row.append(key[0])
        col.append(key[1])
        data.append(val)
    return coo_matrix((data, (row,col)))

Testing the function

In [44]:
a = [[1,2,0],[0,0,3],[1,0,4]]
b = [[0,0,7],[0,0,0],[3,0,2]]
A = coo_matrix(a)
B = coo_matrix(b)

In [45]:
print(mat_mul_coo(A, B).toarray())


[[ 0  0  7]
 [ 9  0  6]
 [12  0 15]]


implement the function `mat_mul_csr` that takes two sparse matrices in `csr` format and returns their product.

In [39]:
def mat_mul_csr(A, B):
    #Dimensions compatiblity
    if A.shape[1] != B.shape[0]:
        raise ValueError("Different shapes")

    # C matrix which will be the resulting matrix
    C_data = []
    C_indices = []
    C_indptr = [0]

    # The row pointers for matrix A
    A_ptrs = {}
    for i in range(A.shape[0]):
        A_ptrs[i] = set(A.indices[A.indptr[i]:A.indptr[i+1]])

    # The column indices for matrix B
    B_cols = {}
    for j in range(B.shape[1]):
        B_cols[j] = set(B.indices[B.indptr[j]:B.indptr[j+1]])

    # The product of the two matrices
    for i in range(A.shape[0]):
        for j in range(B.shape[1]):
            # The product of A[i,:] and B[:,j]
            C_ij = sum([A[i,k] * B[k,j] for k in A_ptrs[i].intersection(B_cols[j])])
            if C_ij != 0:
                C_data.append(C_ij)
                C_indices.append(j)
        C_indptr.append(len(C_data))

    # The resulting matrix in CSR format
    return csr_matrix((C_data, C_indices, C_indptr), shape=(A.shape[0], B.shape[1]))


Testing the function

In [40]:
a = csr_matrix([[7,0,0],[14,0,0],[21,0,0]])
b = csr_matrix([[5,5,0],[7,0,7],[0,9,9]])

In [41]:
print(mat_mul_csr(a, b).toarray())


[[ 35  35   0]
 [ 70  70   0]
 [105 105   0]]


implement a function `solve_lin_sys` that takes a matrix `A` in `csr` format and a vector `b` as a numpy array and solves the system `Ax = b`.

In [46]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.linalg import qr

def solve_lin_sys(A_csr, b):
    # Perform QR decomposition of A
    n = A_csr.shape[0]
    Q, R = qr(A_csr.toarray())

    # R*x = Q.T*b
    y = np.dot(Q.T, b)
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = (y[i] - np.dot(R[i, i+1:], x[i+1:])) / (R[i, i])

    return x

Testing the function

In [58]:
A = csr_matrix([[7, 0, -2], [5, 1, 0], [-3, 0, 1]])
b = np.array([1, 2, 3])


print(solve_lin_sys(A, b))

[  7. -33.  24.]


In [59]:
np.linalg.solve(A.toarray(), b)

array([  7., -33.,  24.])